#Corporación Favorita Grocery Sales Forecasting
The objective is to predict future sales of items in Favorita grocery stores across different regions of Ecuador. So that Reliable predictions can be helpful to optimize inventory management, prevent stockouts, and improve promotion strategies.

#1- Importing Libraries

In [ ]:
!pip install -q pmdarima numpy==1.26.4

In [ ]:
!pip install -q pmdarima

In [ ]:
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model  import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima import auto_arima

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(rc={'figure.figsize':(15, 5)})
sns.set(font_scale=2)
sns.set_style("whitegrid")
sns.set_context("talk")

from sklearn.metrics import r2_score, mean_absolute_error
import pandas as pd
import numpy as np

import os
from tqdm import tqdm
import json

import warnings
# Ignore FutureWarning from the 'sklearn.utils.deprecation' module
warnings.filterwarnings("ignore", category=FutureWarning, module='sklearn.utils.deprecation')

In [ ]:
#mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 2- Data Retreival

In [ ]:
# Select data before April'14
max_date = '2014-04-01'

# Initialize an empty list to hold filtered chunks
filtered_chunks = []

# Define the chunk size (number of rows per chunk)
chunk_size = 10 ** 6

# Read the CSV file in chunks
file_path = '/Users/D065623/Documents/Amna/Time_series_project/Data/'
for chunk in pd.read_csv(file_path + 'train_Guayas_featureEngg.csv', chunksize=chunk_size):

    chunk_filtered = chunk[(chunk['date']<max_date)]
    # Append the filtered chunk to the list
    filtered_chunks.append(chunk_filtered)

    del chunk

# Concatenate all filtered chunks into a single DataFrame
df_filtered = pd.concat(filtered_chunks, ignore_index=True)

# Clean up to free memory
del filtered_chunks

In [ ]:
df = df_filtered.copy()
print(df.shape)
df.head()

(9437752, 11)


,store_nbr,item_nbr,date,id,unit_sales,onpromotion,year,month,day,day_of_week,unit_sales_7d_avg
0,24,105574,2013-01-02,18790.0,12.0,False,2013,1,2,2,2.86605
1,24,105574,2013-01-03,59692.0,1.0,False,2013,1,3,3,2.86605
2,24,105574,2013-01-04,99664.0,3.0,False,2013,1,4,4,2.86605
3,24,105574,2013-01-05,140805.0,4.0,False,2013,1,5,5,2.86605
4,24,105574,2013-01-06,182800.0,7.0,False,2013,1,6,6,2.86605


# 3. Classical Model for Time series

## 3.1- SARIMA + autoarima
SARIMA stands for Seasonal Autoregressive Integrated Moving Average. It is an extension of the ARIMA model that explicitly handles time series with a seasonal component. A SARIMA model is typically denoted as SARIMA(p, d, q)(P, D, Q, m).
- p: Order of the non-seasonal Autoregressive (AR) part.
- d: Order of non-seasonal differencing (Integrated part).
- q: Order of the non-seasonal Moving Average (MA) part.

In addition to these, SARIMA includes seasonal components denoted as (P, D, Q, m):

- P: Order of the seasonal Autoregressive (AR) part.
- D: Order of seasonal differencing (Seasonal Integrated part).
- Q: Order of the seasonal Moving Average (MA) part.
- m: The number of time steps for a single seasonal period (e.g.,for given case 7 for daily data with weekly seasonality).

In [ ]:
def model_evaluation(model, test_size = 90, exog=None):
    '''
    This function evaluates the model on a given test size
    '''
    train_predictions = model.predict()
    if exog is not None:
        forecast = model.get_forecast(test_size, exog = df[-test_size:][exog])
        test_predictions = forecast.predicted_mean

    else:
        try:
            test_predictions = model.forecast(test_size)

        except:
            test_predictions = model.predict(test_size)

    print(len(test_predictions))
    mae = mean_absolute_error(df[-test_size:]['unit_sales'], test_predictions)
    r2 = r2_score(df[-test_size:]['unit_sales'], test_predictions)
    print(f'R2 score {np.round(r2,2)}\nMean absolute error {np.round(mae,2)}')


    #plot
    fig, ax = plt.subplots()
    ax.plot(train_predictions, label = 'Train predictions')
    ax.plot(df['unit_sales'], label = 'Input data')
    ax.plot(test_predictions, label = 'Test predictions')
    plt.legend()
    plt.show()

## 3.2: Model Training per Store Per Item, and forecast their sale

For each Store in Guayas region, top 10 sold items are selected and used to train an Auto-Arima model per item. As there are 10 Stores and 10 item each, hence 100 auto-arima models are trained.

The trained models forecasts per store per item are saved in a dictionary and later exported as Json. This Json file will be used with app.py to show results in the streamlit app.


In [ ]:
unique_stores = df['store_nbr'].unique()

#drop store nbr 29 because of no sales from unique_stores
unique_stores = unique_stores[unique_stores != 29]
print(unique_stores)

[24 26 27 28 30 32 34 35 51 36]


In [ ]:
# Initialize a dictionary to hold forecasts for each store
stores_forecasts = {}

# For each store, groupby item_nbr and aggregate sales to find the top_10 sales items. Each store will have its own top_10 items
for store in unique_stores:
    store_k = str(store)
    stores_forecasts[store_k] = {}

    store_data = df[df['store_nbr'] == store]
    top_items = store_data.groupby('item_nbr')['unit_sales'].sum().nlargest(10).index.tolist()
    stores_forecasts[store_k]['top_items'] = top_items

    # make store top items as key a new dictionatry in stores_forecasts[store]
    for item in top_items:
        item_k = str(item)
        stores_forecasts[store_k][item_k] = {}

#print(stores_forecasts)

In [ ]:
# for each store in unique_stores, prepate the data to be used in Auto ARIMA model,
# and then enrich the stores_forecasts dictionary with the forecasts for each store and item

plotting = False
count = 0

# Number of periods to forecast, for example, 90 for 3 months forecast
n_periods = 15

for store in tqdm(unique_stores):
    store_k = str(store)
    store_data = df[df['store_nbr'] == store]

    for item in stores_forecasts[store_k]['top_items']:
        count += 1
        item_k = str(item)

        # Filter data for the specific item
        item_data = store_data[store_data['item_nbr'] == item]

        item_data_sales = item_data.groupby('date').sum()['unit_sales'].reset_index()
        item_data_sales.set_index('date', inplace=True)
        item_data_sales.index = pd.DatetimeIndex(item_data_sales.index.values, freq='D')

        # Ensure the data is sorted by date
        item_data_sales = item_data_sales.sort_index()

        # split into train and test sets as per split date
        train_data, test_data = item_data_sales.loc[:'2013-12-31'], item_data_sales.loc['2014-01-01':]
        test_data = test_data.head(n_periods)  # Limit test data to n_periods

        if plotting:
            fig, ax = plt.subplots()
            ax.plot(train_data['unit_sales'], label='Train Data')
            ax.plot(test_data['unit_sales'], label='Test Data')
            plt.title(f'Store {store_k}, Item {item_k} Sales Data')
            plt.xlabel('Date')
            plt.ylabel('Unit Sales')
            plt.legend()
            plt.show()


        # Fit the Auto ARIMA model
        model = auto_arima(train_data,
                         start_p=1, start_q=1,
                         test='adf',
                         max_p=25, max_q=25,
                         m=7, #7 is the frequncy of the cycle
                         start_P=0,
                         seasonal=True, #set to seasonal
                         d=0, #order of the non-seasonal differencing
                         D=1, #order of the seasonal differencing
                         trace=False,
                         error_action='ignore',
                         suppress_warnings=True,
                         stepwise=True)


        # Forecast for the next 90 days
        forecast = model.predict(n_periods=n_periods)
        forecasted_data = pd.DataFrame({'date': pd.date_range(start='2014-01-01', periods=n_periods, freq='D'),
                                 'unit_sales': test_data['unit_sales'],
                                 'forcasted_unit_sales': forecast})
        forecasted_data.set_index('date', inplace=True)


        # save the train and test data to the stores_forecasts dictionary (optional)
        stores_forecasts[store_k][item_k]['train_data'] = train_data['unit_sales'].round(2).tolist()
        stores_forecasts[store_k][item_k]['test_data'] = test_data['unit_sales'].round(2).tolist()

        # Store the actual and forecasted data in the dictionary, with upto 2 decimal points
        stores_forecasts[store_k][item_k]['forecast'] = forecasted_data['forcasted_unit_sales'].round(2).tolist()

        #stores_forecasts[store_k][item_k]['forecast'] = forecasted_data['forcasted_unit_sales'].tolist()

        # Optionally, you can also store the confidence intervals of forecasted_data if available
        # Get confidence intervals if available

        try:

            forecast = model.get_forecast(steps=n_periods)
            confidence_intervals = forecast.conf_int()

        except AttributeError:
            # If the model does not support confidence intervals, set to None
            confidence_intervals = None
        except Exception as e:
            confidence_intervals = None

        # Store confidence intervals in the dictionary
        stores_forecasts[store_k][item_k]['confidence_intervals'] = confidence_intervals

        if plotting:
            # Plot the forecasted data
            plt.figure(figsize=(15, 5))
            plt.plot(forecasted_data.index, forecasted_data['unit_sales'], label='Actual')
            plt.plot(forecasted_data.index, forecasted_data['forcasted_unit_sales'], label='Forecast')

            # Plot confidence intervals if available
            if confidence_intervals is not None:
                # Convert confidence intervals to DataFrame for plotting
                ci = pd.DataFrame(confidence_intervals, index=forecasted_data.index)
                # Plot the confidence intervals
                plt.fill_between(ci.index, ci.iloc[:, 0], ci.iloc[:, 1], color='gray', alpha=0.2, label='Confidence Interval')
            else:
                # If confidence intervals are not available, plot a placeholder
                ci = pd.DataFrame(index=forecasted_data.index, data=np.nan, columns=['lower', 'upper'])
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.grid()
            plt.title(f'Store {store_k}, Item {item_k} Forecast')
            plt.xlabel('Date')
            plt.ylabel('Unit Sales')
            plt.legend()
            plt.show()


        # Calculate Bias, MAD, rMAD
        forecasted_data['Bias'] = forecasted_data['unit_sales'] - forecasted_data['forcasted_unit_sales']
        Bias = forecasted_data['Bias'].mean()
        MAD = forecasted_data['Bias'].abs().mean()
        rMAD = forecasted_data['Bias'].abs().mean() / forecasted_data['unit_sales'].mean()

        # calculate the r2 and mae
        r2 = r2_score(forecasted_data['unit_sales'], forecasted_data['forcasted_unit_sales'])
        mae = mean_absolute_error(forecasted_data['unit_sales'], forecasted_data['forcasted_unit_sales'])


        # Store the Bias, MAD, rMAD in the dictionary
        stores_forecasts[store_k][item_k]['Bias'] = Bias.round(2)
        stores_forecasts[store_k][item_k]['MAD'] = MAD.round(2)
        stores_forecasts[store_k][item_k]['rMAD'] = rMAD.round(2)
        stores_forecasts[store_k][item_k]['r2'] = round(r2,2)
        stores_forecasts[store_k][item_k]['mae'] = round(mae,2)

# Model Evaluation
        # Print the Bias, MAD, rMAD
        print(f'{count}. Store {store_k}, Item {item_k} Bias: {Bias.round(2)}, MAD: {MAD.round(2)}, rMAD: {rMAD.round(2)}, r2: {round(r2,2)}, mae: {round(mae,2)}')

        #break
    #break
    print('-'*50)


  0%|          | 0/10 [00:00<?, ?it/s]

1. Store 24, Item 257847 Bias: 35.11, MAD: 87.8, rMAD: 0.42, r2: -0.3, mae: 87.8
2. Store 24, Item 1047679 Bias: -12.96, MAD: 55.37, rMAD: 0.47, r2: -0.61, mae: 55.37
3. Store 24, Item 819932 Bias: -16.74, MAD: 48.63, rMAD: 0.55, r2: -0.55, mae: 48.63
4. Store 24, Item 315176 Bias: 29.06, MAD: 37.33, rMAD: 0.37, r2: -0.12, mae: 37.33
5. Store 24, Item 305080 Bias: -9.84, MAD: 26.26, rMAD: 0.35, r2: -0.54, mae: 26.26
6. Store 24, Item 839362 Bias: -11.1, MAD: 25.99, rMAD: 1.27, r2: 0.29, mae: 25.99
7. Store 24, Item 559870 Bias: -10.69, MAD: 16.76, rMAD: 0.3, r2: -0.39, mae: 16.76
8. Store 24, Item 215352 Bias: 9.55, MAD: 22.59, rMAD: 0.36, r2: -0.1, mae: 22.59
9. Store 24, Item 364606 Bias: -15.37, MAD: 22.5, rMAD: 0.49, r2: -0.78, mae: 22.5


 10%|█         | 1/10 [02:08<19:15, 128.40s/it]

10. Store 24, Item 1037857 Bias: 11.99, MAD: 35.1, rMAD: 0.62, r2: -0.25, mae: 35.1
--------------------------------------------------
11. Store 26, Item 839362 Bias: -5.94, MAD: 31.9, rMAD: 1.19, r2: -0.71, mae: 31.9
12. Store 26, Item 364606 Bias: -21.67, MAD: 25.41, rMAD: 1.17, r2: -6.08, mae: 25.41
13. Store 26, Item 265559 Bias: -24.56, MAD: 32.84, rMAD: 0.96, r2: -3.45, mae: 32.84
14. Store 26, Item 807493 Bias: -6.81, MAD: 16.55, rMAD: 0.66, r2: -0.2, mae: 16.55
15. Store 26, Item 305080 Bias: -27.8, MAD: 31.21, rMAD: 1.31, r2: -7.9, mae: 31.21
16. Store 26, Item 841842 Bias: -6.83, MAD: 16.17, rMAD: 0.55, r2: -0.2, mae: 16.17
17. Store 26, Item 220435 Bias: -10.11, MAD: 12.54, rMAD: 0.56, r2: -1.02, mae: 12.54
18. Store 26, Item 115611 Bias: -15.32, MAD: 15.75, rMAD: 0.65, r2: -3.07, mae: 15.75
19. Store 26, Item 559870 Bias: -17.68, MAD: 20.87, rMAD: 0.72, r2: -1.79, mae: 20.87


 20%|██        | 2/10 [09:59<44:00, 330.07s/it]

...

80. Store 35, Item 1146786 Bias: -7.64, MAD: 14.73, rMAD: 0.65, r2: -0.03, mae: 14.73
--------------------------------------------------
81. Store 51, Item 257847 Bias: -12.1, MAD: 72.88, rMAD: 0.31, r2: -0.11, mae: 72.88
82. Store 51, Item 315176 Bias: 30.48, MAD: 53.68, rMAD: 0.31, r2: 0.3, mae: 53.68
83. Store 51, Item 418235 Bias: -24.41, MAD: 53.8, rMAD: 0.61, r2: -1.2, mae: 53.8
84. Store 51, Item 364738 Bias: 52.11, MAD: 64.21, rMAD: 0.52, r2: 0.03, mae: 64.21
85. Store 51, Item 1047679 Bias: 18.06, MAD: 43.03, rMAD: 0.35, r2: -0.07, mae: 43.03
86. Store 51, Item 819932 Bias: 2.61, MAD: 42.71, rMAD: 0.5, r2: 0.16, mae: 42.71
87. Store 51, Item 305080 Bias: -61.76, MAD: 68.27, rMAD: 0.81, r2: -15.27, mae: 68.27
88. Store 51, Item 1074327 Bias: 13.55, MAD: 28.45, rMAD: 0.29, r2: 0.36, mae: 28.45
89. Store 51, Item 364606 Bias: -26.61, MAD: 28.18, rMAD: 0.44, r2: -2.57, mae: 28.18


 90%|█████████ | 9/10 [41:25<03:59, 239.11s/it]

90. Store 51, Item 559870 Bias: 8.79, MAD: 21.4, rMAD: 0.25, r2: 0.09, mae: 21.4
--------------------------------------------------
91. Store 36, Item 807493 Bias: 2.84, MAD: 15.97, rMAD: 0.22, r2: 0.43, mae: 15.97
92. Store 36, Item 215352 Bias: -8.68, MAD: 19.43, rMAD: 0.33, r2: 0.27, mae: 19.43
93. Store 36, Item 1143685 Bias: 10.68, MAD: 26.73, rMAD: 0.38, r2: 0.07, mae: 26.73
94. Store 36, Item 115894 Bias: 2.57, MAD: 16.43, rMAD: 0.33, r2: -0.19, mae: 16.43
95. Store 36, Item 1158720 Bias: 16.92, MAD: 20.47, rMAD: 0.38, r2: -0.5, mae: 20.47
96. Store 36, Item 839362 Bias: 7.42, MAD: 13.23, rMAD: 0.61, r2: 0.02, mae: 13.23
97. Store 36, Item 1161572 Bias: 6.54, MAD: 13.83, rMAD: 0.33, r2: -0.19, mae: 13.83
98. Store 36, Item 1146786 Bias: 2.3, MAD: 9.31, rMAD: 0.37, r2: -0.3, mae: 9.31
99. Store 36, Item 364606 Bias: -16.94, MAD: 16.94, rMAD: 0.82, r2: -3.31, mae: 16.94


100%|██████████| 10/10 [45:20<00:00, 272.03s/it]

100. Store 36, Item 155500 Bias: -11.01, MAD: 16.21, rMAD: 0.54, r2: -1.77, mae: 16.21
--------------------------------------------------


In [ ]:
def convert_types(obj):
    if isinstance(obj, dict):
        return {k: convert_types(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_types(i) for i in obj]
    elif isinstance(obj, (np.integer, np.int64)):
        return int(obj)
    elif isinstance(obj, (np.floating, np.float64)):
        return float(obj)
    else:
        return obj

In [ ]:
# save the stores_forecasts dictionary to a json file
json_save_path = '/Users/D065623/Documents/Amna/Time_series_project/Forcasting_app/forecasts/stores_forecasts_15.json'
with open(json_save_path, 'w') as f:
    json.dump(convert_types(stores_forecasts), f)
print(f'Stores forecasts saved to {json_save_path}')

Stores forecasts saved to /Users/D065623/Documents/Amna/Time_series_project/Forcasting_app/forecasts/stores_forecasts_15.json


### 3.3- Load json

In [ ]:
# load the stores_forecasts dictionary from the json file
json_save_path = '/Users/D065623/Documents/Amna/Time_series_project/Forcasting_app/forecasts/stores_forecasts_15.json'
with open(json_save_path, 'r') as f:
     stores_forecasts = json.load(f)


In [ ]:
# create a key 'date' in the stores_forecasts dictionary. and populate it with two further keys 'train_data_dates' and 'test_data_dates'. Then save the unique dates from the df dataframe to 'train_data_dates' and 'test_data_dates'
stores_forecasts['date'] = {}
unique_dates = df['date'].unique()

# select unique dates from unique_dates list till '2013-12-31' in a new list
unique_dates_df = pd.DataFrame(unique_dates, columns=['date'])
train_data_dates = unique_dates_df[unique_dates_df['date'] < '2014-01-01']
test_data_dates = unique_dates_df[unique_dates_df['date'] >= '2014-01-01']

# limit the test_data_dates to 15 days
test_data_dates = test_data_dates.head(n_periods)

stores_forecasts['date']['train_data_dates'] = train_data_dates['date'].tolist()
stores_forecasts['date']['test_data_dates'] = test_data_dates['date'].tolist()


In [ ]:
# update the stores_forecast dictionary to json file 'stores_forecasts_3.json'
json_save_path = '/Users/D065623/Documents/Amna/Time_series_project/Forcasting_app/forecasts/stores_forecasts_15.json'
with open(json_save_path, 'w') as f:
    json.dump(convert_types(stores_forecasts), f)
print(f'Stores forecasts saved to {json_save_path}')

Stores forecasts saved to /Users/D065623/Documents/Amna/Time_series_project/Forcasting_app/forecasts/stores_forecasts_15.json
